# improts

In [2]:
# imports
import requests
import pandas as pd
from notion_client import Client
import datetime
import json
import time

function definition

In [3]:
# function that takes a property value and a property type and returns the text content from notion database
def get_text_content(prop_value, prop_type='rich_text'):
    if not prop_value:
        return ''
    try:
        if prop_type == 'date':
            return prop_value.get('date', {}).get('start', '')
        if prop_type == 'rich_text':
            texts = prop_value.get('rich_text', [])
            if not texts:
                return ''
            first_text = texts[0].get('text', {})
            return first_text.get('content', '')
        elif prop_type == 'title':
            titles = prop_value.get('title', [])
            if not titles:
                return ''
            first_title = titles[0].get('text', {})
            return first_title.get('content', '')
        elif prop_type == 'email':
            return prop_value.get('email', '')
    except (AttributeError, IndexError, KeyError):
        return ''
    return ''

def get_follow_up_date_groq(transcription, call_date):
    # Call to Groq API
    headers = {
        "Authorization": f"Bearer {GROQ_API_KEY}",
        "Content-Type": "application/json"
    }
    payload = {
        "model": "deepseek-r1-distill-llama-70b",
        "messages": [
            {"role": "system", "content": "You are an AI assistant tasked with analyzing call transcripts and suggesting specific follow-up dates, please resonse with ISO 8601 time and date format, for example: \"2025-02-06 12:00:00\", do not add any other text"},
            {"role": "user", "content": f"Based on this call transcript, suggest a specific follow-up date: {transcription}, the time of the call is{call_date}"}
        ]
    }
    groq_response = requests.post(groq_url, json=payload, headers=headers)
    return groq_response.json()['choices'][0]['message']['content'].strip().split('\n')[-1]
    
def update_data_base_after_cold_call(notion_raw_id, call_id):
    
    #collect call id data
    call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)

    # update database with : call id, last call date, next follow up date, call transcription, lead status
    
    call_info_dict = {}
    
    # collect time of call
    call_info_dict["Last call date"] = call_id_response["created_at"]
    
    #collect call id
    call_info_dict["Last call ID"] = call_id
    
    # if cosutmer is interested:
    if "Interested" in call_id_response["pathway_tags"]:
        # collect all information needed and update notion DB
        
        # collect costumer email
        call_info_dict["Main contact Email"] = call_id_response["variables"]["email"]
        # collect follow up date from transcription
        call_info_dict["Next Follow Up Date"]  = get_follow_up_date_groq(call_id_response['concatenated_transcript'], call_id_response["created_at"])
        # collect lead status
        call_info_dict["Status of lead"] = call_id_response["pathway_tags"][0]
        # collect cold call transcription
        call_info_dict["cold call transcription"] = call_id_response["concatenated_transcript"]   
        
        # update notion table
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'cold call transcription': {
                    'rich_text': [
                        {
                            'text': {
                                'content': call_info_dict["cold call transcription"]
                            }
                        }
                    ]
                },
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                },
                'Main contact Email': {
                    'email': call_info_dict["Main contact Email"]
                },
                'Next Follow Up Date': {
                    'date': {
                        'start': call_info_dict["Next Follow Up Date"]
                    }
                }
            }
        )
    # if cosutmer did not answer
    elif not call_id_response["pathway_tags"]:
        #collect Follow Up Date and update notion DB

        # update Next Follow Up Date to 7 days after the call
        call_info_dict["Next Follow Up Date"] = (pd.to_datetime(call_id_response["created_at"]) + pd.Timedelta(days=1)).strftime('%Y-%m-%d %H:%M:%S')
        
        call_info_dict["Status of lead"] = 'Did not answered'
        
        # update notion table with new follow up time and lead status
         notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                },
                'Next Follow Up Date': {
                    'date': {
                        'start': call_info_dict["Next Follow Up Date"]
                    }
                }
            }
        )
        
    elif "Not interested" in call_id_response["pathway_tags"]:
        #collect trascription and update notion database
        
        # collect cold call transcription
        call_info_dict["cold call transcription"] = call_id_response["concatenated_transcript"] 
          
        call_info_dict["Status of lead"] = 'Not interested'
        
        # update notion table with new follow up time and lead status
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'cold call transcription': {
                    'rich_text': [
                        {
                            'text': {
                                'content': call_info_dict["cold call transcription"]
                            }
                        }
                    ]
                },
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                }
            }
        )
        
    elif "Asked for follow up" in call_id_response["pathway_tags"]:
        #collect follow up date and update notion DB
        
        # collect cold call transcription   
        call_info_dict["Next Follow Up Date"]  = get_follow_up_date_groq(call_id_response['concatenated_transcript'], call_id_response["created_at"])
        
        call_info_dict["Status of lead"] = 'Asked for follow up'
        
        # update notion table with new follow up time and lead status        
        notion.pages.update(
            page_id=notion_raw_id,
            properties={
                'Status of lead': {
                    'status': {
                        'name': call_info_dict["Status of lead"]
                    }
                },
                'Next Follow Up Date': {
                    'date': {
                        'start': call_info_dict["Next Follow Up Date"]
                    }
                }
            }
        )
        
    else:
        # call got hang out for some reason, need to think what to do
        pass
            
def send_communication(call_id, Customer_phone_number): 
    pass           
            
            
                       
    

Setup

In [4]:
# API setup
BLEND_API_KEY = "org_ba2e4ccfb75e56afc088d9804df57d2623542e8bbd3de2c02bfcb0024daa778c1850bba9de94a2d1ec6a69"
Blend_url = "https://api.bland.ai/v1/calls"

#groq setup
GROQ_API_KEY = "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"
groq_url = "https://api.groq.com/openai/v1/chat/completions"

# Initialize Notion client
NOTION_TOKEN = "ntn_S6159294934albrajfceBHL4szrrrMllKAcFNUGM62v7JI"
DATABASE_ID = "18316f1f61d680a2921bd08b8c62f895"
notion = Client(auth=NOTION_TOKEN)

#call script setup
COLD_CALL_SCRIPT = "6a5a0412-6481-4533-b560-cf72283e956b"
CLOSER_CALL_SCRIPT = "29e7ef67-4d36-4d15-aa09-0a38642fea26"


# call general information
System_phone_number = "null"
Interruption_Threshold_in_ms = 100
LLM_temperature = 1
Agent_name = "Christin"
Cold_Agent_voice = "Public - June 2978"

Pull data

In [5]:
# create empty data arrey
data = []

# call notion API 
response = notion.databases.query(database_id=DATABASE_ID)

# iterate through each page and collect data
for page in response['results']:
    # get page properties
    properties = page.get('properties', {})
    # select the fields we want to extract
    row = {
        'notion raw id': page['id'],
        'customer name': get_text_content(properties.get('Main contact name')),
        'Phone': properties.get('Contact 1 phone', {}).get('Main contact phone', ''),
        'Email': get_text_content(properties.get('Main contact Email'), 'email'),
        'home address': get_text_content(properties.get('Address')),
        'Next Follow Up Date': properties.get('Next Follow Up Date', {}).get('date', {}).get('start', '') if properties.get('Next Follow Up Date') and properties.get('Next Follow Up Date').get('date') else '',
        'Claim': properties.get('Status of Claim', {}).get('status', {}).get('name', ''),
        'Lead': properties.get('Status of lead', {}).get('status', {}).get('name', ''),   
        'call id' : get_text_content(properties.get('Last call ID')),
        'First_call_transcription': get_text_content(properties.get('cold call transcription'))
    }
    # append data to the data arrey
    data.append(row)
    
# convert data to a dataframe   
df = pd.DataFrame(data)

create data frame for cold calling for new leads

In [7]:
# filter df for "New" leads with no follow up data, or "Did not answered" with follow up date which is less then now

cold_call_df = df[(df['Lead'] == 'New') & (df['Next Follow Up Date'] == '') |
                   (df['Lead'] == 'Did not answered') & (pd.to_datetime(df['Next Follow Up Date']).dt.tz_localize(None) < pd.Timestamp(datetime.datetime.now()).tz_localize(None))]
cold_call_df


,notion raw id,customer name,Phone,Email,home address,Next Follow Up Date,Claim,Lead,call id,First_call_transcription
0,18516f1f-61d6-8056-acb8-d896832942bd,test,,sda@askld.com,test1,,Not approved by client,New,123aa,
1,18516f1f-61d6-80cc-8477-d40b7aa4ccd1,test,,sda@askld.com,test1,2025-02-06T00:00:00.000+02:00,Not approved by client,Did not answered,,


cold call all new leads and update data base for each lead

In [ ]:
# iterate rows and collect user data
for index, row in cold_call_df.iterrows():
    Customer_phone_number = row['Phone']
    Customer_name = row['customer name']
    Home_address = row['home address']
    notion_raw_id = row['notion raw id']

    # cold call setup *need to add system phone number
    headers = {
    'Authorization': BLEND_API_KEY,
    "Content-Type": "application/json"
    }
    data = {
        "phone_number": Customer_phone_number,
        "task": "",
        "model": "enhanced",
        "language": "en",
        "voice": Cold_Agent_voice,
        "voice_settings": {},
        "pathway_id": COLD_CALL_SCRIPT,
        "local_dialing": False,
        "max_duration": "12",
        "answered_by_enabled": False,
        "wait_for_greeting": True,
        "noise_cancellation": True,
        "ignore_button_press": True,
        "record": False,
        "amd": False,
        "interruption_threshold": Interruption_Threshold_in_ms,
        "voicemail_message": "test",
        "temperature": LLM_temperature,
        "transfer_list": {},
        "pronunciation_guide": [],
        "request_data": {
        "customer name": Customer_name,
        "home address": Home_address,
        "agent name": Agent_name,
        },
        "retry": {
        "wait": 240,
        "voicemail_action": "hangup",
        "voicemail_message": ""
        },
        "dynamic_data": [],
        "analysis_schema": {},
        "calendly": {},
        "timezone": "America/New_York"
    }
    
    # send call
    send_call_response = json.loads(requests.request("POST", url, json=data, headers=headers).text)
    
    # print status
    print(send_call_response.text)
    
    # collect call id
    call_id = send_call_response["call_id"]
    
    call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
    # wait for call to end
    while call_id_response["completed"] == False:
        time.sleep(3)
        print("Call is still active")
        call_id_response = json.loads(requests.request("GET", url+"/"+call_id, headers=headers).text)
        
    # wait for tags to update
    time.sleep(3)
    
    # update database based on results
    update_data_base_after_cold_call(notion_raw_id,call_id)

    # send correct mail
    send_communication(call_id, Customer_phone_number): 
    
    
    
    
    
    
    
    
    
    
    